In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset('trivia_qa', name = 'rc')  # name = 'rc' to get reading comprehension version (with full texts)

Reusing dataset trivia_qa (/Users/IdaRiis/.cache/huggingface/datasets/trivia_qa/rc/1.1.0/e734e28133f4d9a353af322aa52b9f266f6f27cbf2f072690a1694e577546b0d)


In [3]:
train = dataset['train']
test = dataset['test']
validation = dataset['validation']

In [5]:
tmp_data = train.map(lambda example: {'wiki_text': example['entity_pages']['wiki_context'], 'answer': example['answer']['normalized_value']}, remove_columns=['search_results', 'question_source', 'answer'])

 63%|██████▎   | 86999/138384 [09:51<05:49, 146.99ex/s]


OSError: [Errno 28] Error writing bytes to file. Detail: [errno 28] No space left on device

In [7]:
tmp_data = train.select(range(11))
tmp_data = tmp_data.map(lambda example: {'wiki_text': example['entity_pages']['wiki_context'], 'answer': example['answer']['normalized_value']}, remove_columns=['search_results', 'question_source', 'answer', 'entity_pages'])

AttributeError: 'dict' object has no attribute 'map'

In [31]:
len(tmp_data)

11

In [44]:
tmp_data.features

{'answer': Value(dtype='string', id=None),
 'question': Value(dtype='string', id=None),
 'question_id': Value(dtype='string', id=None),
 'wiki_text': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [46]:
tmp_data = tmp_data.map(lambda example: {'seq_len': len(example['wiki_text'])})

100%|██████████| 11/11 [00:00<00:00, 1124.51ex/s]


In [48]:
tmp_data['seq_len']

[0, 2, 0, 3, 1, 1, 2, 1, 1, 1, 1]

In [52]:
tmp_data['question'][1]

'Where in England was Dame Judi Dench born?'

In [58]:
tmp_data = tmp_data.filter(lambda example: len(example['wiki_text']) == 1)

Loading cached processed dataset at /Users/IdaRiis/.cache/huggingface/datasets/trivia_qa/rc/1.1.0/e734e28133f4d9a353af322aa52b9f266f6f27cbf2f072690a1694e577546b0d/cache-46e45beffe321c8b.arrow


In [60]:
tmp_data = tmp_data.map(lambda example: {'wiki_text_lower': example['wiki_text'][0].lower()})

100%|██████████| 6/6 [00:00<00:00, 387.20ex/s]


In [63]:
tmp_data['wiki_text_lower']

s of the members of a boys\' choir, led by the red-headed jack merridew. ralph asserts three primary goals: to have fun, survive, and to maintain a smoke signal that could alert passing ships to their presence on the island. the boys declare that whoever holds the conch shall also be able to speak at their formal gatherings and receive the attentive silence of the larger group.\n\njack organises his choir group into a hunting party responsible for discovering a food source. ralph, jack, and a quiet, dreamy boy named simon soon form a loose triumvirate of leaders. though he is ralph\'s only confidant, piggy is quickly made into an outcast by his fellow "biguns" (older boys) and becomes an unwilling source of laughs for the other children. simon, in addition to supervising the project of constructing shelters, feels an instinctive need to protect the "littluns" (younger boys).\n\nthe semblance of order quickly deteriorates as the majority of the boys turn idle, giving little aid in build

In [64]:
type(tmp_data['wiki_text'])

list

In [65]:
from nltk.tokenize import RegexpTokenizer

In [69]:
tokenizer = RegexpTokenizer(r'\w+')

# Remove punctuation
tmp_data = tmp_data.map(lambda example: {'tokens': tokenizer.tokenize(example['wiki_text_lower']), 'question_token': tokenizer.tokenize(example['question'].lower())})

100%|██████████| 6/6 [00:00<00:00, 64.80ex/s]


In [71]:
tmp_data['question_token'][0]

['which', 'city', 'does', 'david', 'soul', 'come', 'from']

In [72]:
tmp_data.save_to_disk('/Users/IdaRiis/Dropbox/DTU/MMC 2. semester/02456 Deep Learning/deep_learning_project/Data/Processed')